In [232]:
import cudf
import numpy as np

In [75]:
''' 
+++++ UTILITY ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code for the customer that owns this utility
    customer_code --> String --> Combined with user_code provides a unique identifier for the utility. Even this field is anonymized
    city --> String --> City where the utility is located
    address --> String --> (Anonymized) address of the utility location
'''

' \n+++++ UTILITY ATTRIBUTES +++++\n    \n    user_code --> String --> (Anonymized) code for the customer that owns this utility\n    customer_code --> String --> Combined with user_code provides a unique identifier for the utility. Even this field is anonymized\n    city --> String --> City where the utility is located\n    address --> String --> (Anonymized) address of the utility location\n'

In [76]:
''' 
+++++ CUSTOMER ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code that identifies the customer
    nominative --> String --> (Anonymized) customer name
    sex --> String --> Sex of the customer. It could be ‘M’, ‘F’, ‘P’, with ‘P’ denoting that the customer is a commercial activity (VAT number)
    age --> Int --> Age of the customer, set to null for commercial activities (sex = ‘P’). Its value must be >= 18
'''

' \n+++++ CUSTOMER ATTRIBUTES +++++\n    \n    user_code --> String --> (Anonymized) code that identifies the customer\n    nominative --> String --> (Anonymized) customer name\n    sex --> String --> Sex of the customer. It could be ‘M’, ‘F’, ‘P’, with ‘P’ denoting that the customer is a commercial activity (VAT number)\n    age --> Int --> Age of the customer, set to null for commercial activities (sex = ‘P’). Its value must be >= 18\n'

In [77]:
''' 
+++++ INVOICE ATTRIBUTES +++++
    
    bill_id --> Int --> Invoice identifier
    F1_kWh --> Float --> kWh of electricity consumed in the F1 time slot
    F2_kWh --> Float --> kWh of electricity consumed in the F2 time slot
    F3_kWh --> Float --> kWh of electricity consumed in the F3 time slot 
    date --> Date --> Start date
    light_start_date --> Date --> Start date of electricity invoice
    light_end_date --> Date --> End date of electricity invoice
    tv --> Float --> Television fee to pay
    gas_amount --> Float --> Gas fee to pay
    gas_average_cost --> Float --> Average cost of gas
    light_average_cost --> Float --> Average cost of electricity
    emission_date --> Date --> Emission date
    supply_type --> String --> Supply type (‘light’, ‘gas’, ‘gas and light’)
    gas_start_date --> Date --> Start date of gas invoice
    gas_end_date --> Date --> End date of gas invoice
    extra_fees --> Float --> Extra fees to pay
    gas_consumption --> Float --> Consumed gas
    light_consumption --> Float --> Consumed electricity
    gas_offer --> Float --> Name of the subscribed gas plan (anonymized)
    light_offer_type --> String --> Kind of plan for the electricity (‘single zone’, ‘bizone’, etc.)
    light_offer --> String --> Name of the subscribed electricity plan (anonymized)
    total_amount --> Float --> gas_amount + light_amount + extra_fees
    howmuch_pay --> Float --> Overall amount to pay, computed as total_amount + tv
    light_amount --> Float --> Amount to pay for the electricity
    average_unit_light_cost --> Float --> Average cost for electricity
    average_light_bill_cost --> Float --> Average cost for the electricity invoice
    average_unit_gas_cost --> Float --> Average cost for gas
    average_gas_bill_cost --> Float --> Average cost for the gas invoice
    billing_frequency --> String --> Billing frequency (‘monthly’, ‘quarterly’, etc.)
    bill_type --> String --> Kind of invoice (False means a “standard bill”)
    gas_system_charges --> Float --> Extra gas fees
    light_system_charges --> Float --> Extra electricity fees
    gas_material_cost --> Float --> Costs for gas
    light_transport_cost --> Float --> Extra electricity fees
    gas_transport_cost --> Float --> Extra gas fees
    light_material_cost --> Float --> Costs for electricity
'''

' \n+++++ INVOICE ATTRIBUTES +++++\n    \n    bill_id --> Int --> Invoice identifier\n    F1_kWh --> Float --> kWh of electricity consumed in the F1 time slot\n    F2_kWh --> Float --> kWh of electricity consumed in the F2 time slot\n    F3_kWh --> Float --> kWh of electricity consumed in the F3 time slot \n    date --> Date --> Start date\n    light_start_date --> Date --> Start date of electricity invoice\n    light_end_date --> Date --> End date of electricity invoice\n    tv --> Float --> Television fee to pay\n    gas_amount --> Float --> Gas fee to pay\n    gas_average_cost --> Float --> Average cost of gas\n    light_average_cost --> Float --> Average cost of electricity\n    emission_date --> Date --> Emission date\n    supply_type --> String --> Supply type (‘light’, ‘gas’, ‘gas and light’)\n    gas_start_date --> Date --> Start date of gas invoice\n    gas_end_date --> Date --> End date of gas invoice\n    extra_fees --> Float --> Extra fees to pay\n    gas_consumption --> Fl

In [80]:
df.columns

Index(['bill_id', 'F1_kWh', 'F2_kWh', 'F3_kWh', 'city', 'address',
       'nominative', 'sex', 'age', 'user_code', 'date', 'light_start_date',
       'light_end_date', 'tv', 'gas_amount', 'gas_average_cost',
       'light_average_cost', 'emission_date', 'supply_type', 'gas_start_date',
       'gas_end_date', 'extra_fees', 'gas_consumption', 'light_consumption',
       'gas_offer', 'light_offer_type', 'light_offer', 'howmuch_pay',
       'total_amount', 'light_amount', 'average_unit_light_cost',
       'average_light_bill_cost', 'average_unit_gas_cost',
       'average_gas_bill_cost', 'customer_code', 'billing_frequency',
       'bill_type', 'gas_system_charges', 'light_system_charges',
       'gas_material_cost', 'light_transport_cost', 'gas_transport_cost',
       'light_material_cost'],
      dtype='object')

In [83]:
df[['supply_type', 'gas_consumption', 'light_consumption']]
# --> supply_type indica di quale utility riguarda l'invoice

,supply_type,gas_consumption,light_consumption
npartitions=5,,,
,object,float64,float64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [82]:
df['bill_id'].unique()

<dask_cudf.Series | 21 tasks | 1 npartitions>

In [84]:
df['user_code'].unique()

<dask_cudf.Series | 21 tasks | 1 npartitions>

In [85]:
df['sex'].unique()

<dask_cudf.Series | 21 tasks | 1 npartitions>

In [162]:
df['age'].unique()

0      0
1      1
2      2
3     21
4     22
      ..
76    94
77    95
78    96
79    97
80    98
Name: age, Length: 81, dtype: int64

In [ ]:
''' 
+++++ DATA INGESTION +++++
    • Read data from its source
    • Study how to deal with data that doesn’t fit in memory (!)
    • Locate missing values
    • Locate outliers
    • Sort data
'''

In [233]:
from base import *
base = BaseDfBench()

In [234]:
# READ DATA FROM ITS SOURCE
# --- remove _sample one day

# We can use bill_id as the index for our dataset --> meglio di no
# base.load_dataset('/data/invoices_sample.csv', 'csv', index_col='bill_id')

base.load_dataset('/data/invoices_sample.csv', 'csv')
df = base.get_df()
columns = base.get_columns()

In [235]:
df

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,CIPRESSA,C23B8FC235DF5479FB28D81E827A59B819BB0748,0298E5E66B5653CDAACADD156261A2A916A56C7E,F,81,63D286C61D20D76E9C9317BEB8644D5EE45134E4,...,<NA>,5AE3987CBA311BCAADC6274D28A7FB14B13B53C3,<NA>,False,-0.06,<NA>,0.06,<NA>,-0.2,<NA>
1,1,81.0,62.0,76.0,ROCCAVIVARA,CBE6B021F41D589269FBC724C131CBFC9905D950,FDAE2B1E0934BB76255A727CC9F3F00AA39982D9,M,38,8053C7982DE8A18746A8F9F0D1DC4B0C33CAC0FA,...,<NA>,5F88D510670063B20E789E6453BC2F3FB0BB67F7,<NA>,False,<NA>,9.15,<NA>,5.68,<NA>,23.13
2,2,0.0,0.0,0.0,VIDRACCO,BFAEB566B3BB78B9ABD4F86DB3A78BFD8489013B,FEBE7F29FB854C477AFA7073C70F0E6EA81C6B00,M,53,4F053D54B9F1B6FE565498E7FA726501FD8FDD18,...,<NA>,D4B8928760E729127EB4EB532C86127325FBA468,<NA>,False,1.24,<NA>,-10.13,<NA>,-7.13,<NA>
3,3,0.0,0.0,0.0,MEZZOLOMBARDO,CBD30C382C72ED4A3D9DC9D11C13C155930E7C66,9220A3E83BDCE622F70A1456C7898687FD3EDAC8,M,50,115C7D34A79C66AFCDB4EB65262595B197B0F861,...,"1,08 €/smc",BD4DEF66D7EF2D3D683D9C381D2359AD24B41D9C,<NA>,False,-0.88,<NA>,23.68,<NA>,17.66,<NA>
4,4,0.0,0.0,0.0,PINETO,4D17C711CADED6C1E9BD459088B4B80F24575FE7,642C21354EF6017D13220517E2A25D4A0442D4C5,M,65,C7734B97A212004CEA25956D8FDFEF068CD2B8F6,...,"0,86 €/smc",C0622AFD22384A4B54CD1ACAF7D988B89853A5AC,<NA>,False,14.71,<NA>,141.57,<NA>,63.59,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,26.0,25.0,33.0,CINTANO,2D244E5D90065985546AEA4D52F54C9705B8038B,102CADC9FC8C745FE3B8C27F7267E07C9691728A,M,55,6D3FDD0A2EEEE1A2C1AC23E6C316048DD2CCE859,...,<NA>,53D29ACDEB11F45D4F305A4D00E5F03C1F2CB674,monthly,False,<NA>,13.08,<NA>,7.62,<NA>,15.65
1996,1996,71.0,61.0,88.0,TREZZONE,A01243282C24F561D6B54F1C1FCB45AE80F75B0A,9C3B5B4093FC99569E5BBA918A977681A46F52DF,M,48,52315237533E1A6504E3795EAC1D5586744C62FD,...,<NA>,2B8D02A2AD117EC401BCEF3C0651FEEBB6609D1C,monthly,False,<NA>,9.2,<NA>,8.76,<NA>,25.8
1997,1997,37.0,71.0,100.0,RIPATRANSONE,469AB252FF0A2C60B32D0337C933E3565E88C0C2,1B451B6AA312EFF93372D44EC6C24C0110A628A1,F,32,2777855F19667FBFEDEAEF181E7ABC8584262127,...,<NA>,A1BFA0E4833E27AB398F4420414E037D250F0A7E,monthly,False,<NA>,8.7,<NA>,8.66,<NA>,24.14
1998,1998,74.0,78.0,92.0,MONSELICE,9E24BB26322BD95D11E26DFB80AA1168C9661FDC,F6BADD2456E283A8DCA6F3FF640724072108BAEF,F,54,8DC8820FB50005ED9CC4E7C187B9703460941856,...,<NA>,DED078288677E4E8A15874405E79C26AC78747B0,monthly,False,<NA>,10.21,<NA>,8.96,<NA>,30.43


In [236]:
# STUDY HOW TO DEAL WITH DATA THAT DON'T FIT IN MEMORY (!)
# Bella storia
!du -sh '/data/invoices_sample.csv'

908K	/data/invoices_sample.csv


In [237]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from dask.utils import parse_bytes
import cudf
import dask_cudf

In [238]:
cluster = LocalCUDACluster(
    CUDA_VISIBLE_DEVICES="0",
    rmm_pool_size=parse_bytes("15GB"), # This GPU has 16GB of memory
    device_memory_limit=parse_bytes("10GB"),
)
client = Client(cluster)
client

/opt/conda/envs/rapids/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42547 instead
  f"Port {expected} is already in use.\n"


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:42547/status,
Dashboard: http://127.0.0.1:42547/status,Workers: 1
Total threads: 1,Total memory: 187.40 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34427,Workers: 1
Dashboard: http://127.0.0.1:42547/status,Total threads: 1
Started: Just now,Total memory: 187.40 GiB
Comm: tcp://127.0.0.1:42655,Total threads: 1
Dashboard: http://127.0.0.1:39239/status,Memory: 187.40 GiB
Nanny: tcp://127.0.0.1:38315,


In [94]:
%%time
ddf = dask_cudf.read_csv("/data/invoices.csv", chunksize="1 GiB")
print(ddf.npartitions)
type(ddf)

5
CPU times: user 41.8 ms, sys: 16.8 ms, total: 58.7 ms
Wall time: 47.6 ms


dask_cudf.core.DataFrame

In [95]:
ddf.tail()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
1654276,2512812,0.0,0.0,0.0,CASALZUIGNO,FE1047ECA6069845A48B36ABAD2A725C545A3ACD,59801133A00E8156E2FCB182350344C222C625E9,M,66,CCC653F93977E0948C0E71592DC0B35D8088BF80,...,"1,23 €/smc",B90850838689F4530C6B2403ABF9D6A7DB4A6C18,bimester,False,-2.09,<NA>,41.72,<NA>,34.95,<NA>
1654277,2512813,0.0,0.0,0.0,ACQUANEGRA CREMONESE,31432A91213A1877995BE91793761AB04AB2C1D9,ED3368875442BC333089F2EF36CEF71865954BCC,F,46,CEF01180CFC826BD1B1D940A77532A09A4E13761,...,"1,29 €/smc",6CCC81E05B7ED7401945CB8FB42126B242AB443C,bimester,False,-1.10,<NA>,41.64,<NA>,34.03,<NA>
1654278,2512814,0.0,0.0,0.0,SAN LUPO,B3260B1FAEE19CB253D0B27B10CDCE0F40606AA4,959F1C15E6FD24DF29439A2B4683EC280BB95466,M,45,255D0B199EAAC6E72B483D38E4EA85C643109633,...,"1,58 €/smc",5DC1024A436D07BDF4C4853A752CF89775C3F295,bimester,False,-3.83,<NA>,30.81,<NA>,21.25,<NA>
1654279,2512815,0.0,0.0,0.0,VERNATE,9AADECE289CFA1A60FAE6F30354B16DEEE1644A0,D3FD778C3F95AE2DAB5B9A1FE11FC14728D85164,M,64,A6E6C0A1A17CE15A4A822F516E5F6B1E8CD68B23,...,"1,09 €/smc",454279C1315FE7D78684138FB9998240B99D86DC,bimester,False,-0.22,<NA>,67.14,<NA>,27.61,<NA>
1654280,2512816,0.0,0.0,0.0,AVERSA,5753A0290AC72B2812EC56982D91DAFF45301262,07689D228584CD380A53CF3DC13122F0E634E1BF,M,33,9DF4530117723DCCA54A9502AD31BAD43FF1D709,...,"1,13 €/smc",74999DF227AE1574CF05B90DEC8D63207C552ABE,bimester,False,-0.83,<NA>,44.71,<NA>,25.14,<NA>


In [96]:
ddf.head()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,CIPRESSA,C23B8FC235DF5479FB28D81E827A59B819BB0748,0298E5E66B5653CDAACADD156261A2A916A56C7E,F,81,63D286C61D20D76E9C9317BEB8644D5EE45134E4,...,<NA>,5AE3987CBA311BCAADC6274D28A7FB14B13B53C3,<NA>,False,-0.06,<NA>,0.06,<NA>,-0.2,<NA>
1,1,81.0,62.0,76.0,ROCCAVIVARA,CBE6B021F41D589269FBC724C131CBFC9905D950,FDAE2B1E0934BB76255A727CC9F3F00AA39982D9,M,38,8053C7982DE8A18746A8F9F0D1DC4B0C33CAC0FA,...,<NA>,5F88D510670063B20E789E6453BC2F3FB0BB67F7,<NA>,False,<NA>,9.15,<NA>,5.68,<NA>,23.13
2,2,0.0,0.0,0.0,VIDRACCO,BFAEB566B3BB78B9ABD4F86DB3A78BFD8489013B,FEBE7F29FB854C477AFA7073C70F0E6EA81C6B00,M,53,4F053D54B9F1B6FE565498E7FA726501FD8FDD18,...,<NA>,D4B8928760E729127EB4EB532C86127325FBA468,<NA>,False,1.24,<NA>,-10.13,<NA>,-7.13,<NA>
3,3,0.0,0.0,0.0,MEZZOLOMBARDO,CBD30C382C72ED4A3D9DC9D11C13C155930E7C66,9220A3E83BDCE622F70A1456C7898687FD3EDAC8,M,50,115C7D34A79C66AFCDB4EB65262595B197B0F861,...,"1,08 €/smc",BD4DEF66D7EF2D3D683D9C381D2359AD24B41D9C,<NA>,False,-0.88,<NA>,23.68,<NA>,17.66,<NA>
4,4,0.0,0.0,0.0,PINETO,4D17C711CADED6C1E9BD459088B4B80F24575FE7,642C21354EF6017D13220517E2A25D4A0442D4C5,M,65,C7734B97A212004CEA25956D8FDFEF068CD2B8F6,...,"0,86 €/smc",C0622AFD22384A4B54CD1ACAF7D988B89853A5AC,<NA>,False,14.71,<NA>,141.57,<NA>,63.59,<NA>


In [97]:
df = ddf.loc['0':'4']
df

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_start_date,gas_end_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
npartitions=5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,float64,float64,float64,object,object,object,object,int64,object,object,object,object,object,object,object,float64,object,object,object,object,object,float64,float64,uint64,object,uint64,object,object,object,object,object,object,object,object,int8,bool,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [98]:
# LOCATE MISSING VALUES
rows = ddf.shape[0]
wait(ddf.shape[0])
print("Total rows", rows)

# These are the rows with missing values
for c in columns:
    if (not base.locate_null_values(c).empty):
        print(c, "\t Affected rows:", base.locate_null_values(c).shape[0])
    #else:
        #print(c, "\t Qui no missing values")

Total rows Delayed('int-613f21d3-296b-46d8-8398-cd349a878f69')
address 	 Affected rows: 4
nominative 	 Affected rows: 4
gas_amount 	 Affected rows: 1120
gas_average_cost 	 Affected rows: 1496
light_average_cost 	 Affected rows: 1122
gas_consumption 	 Affected rows: 1120
light_consumption 	 Affected rows: 744
light_amount 	 Affected rows: 744
average_unit_light_cost 	 Affected rows: 1115
average_light_bill_cost 	 Affected rows: 1122
average_unit_gas_cost 	 Affected rows: 1496
average_gas_bill_cost 	 Affected rows: 1494
billing_frequency 	 Affected rows: 1569
gas_system_charges 	 Affected rows: 1155
light_system_charges 	 Affected rows: 776
gas_material_cost 	 Affected rows: 1155
light_transport_cost 	 Affected rows: 775
gas_transport_cost 	 Affected rows: 1155
light_material_cost 	 Affected rows: 775


In [99]:
base.drop_duplicates()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,CIPRESSA,C23B8FC235DF5479FB28D81E827A59B819BB0748,0298E5E66B5653CDAACADD156261A2A916A56C7E,F,81,63D286C61D20D76E9C9317BEB8644D5EE45134E4,...,<NA>,5AE3987CBA311BCAADC6274D28A7FB14B13B53C3,<NA>,False,-0.06,<NA>,0.06,<NA>,-0.2,<NA>
1,1,81.0,62.0,76.0,ROCCAVIVARA,CBE6B021F41D589269FBC724C131CBFC9905D950,FDAE2B1E0934BB76255A727CC9F3F00AA39982D9,M,38,8053C7982DE8A18746A8F9F0D1DC4B0C33CAC0FA,...,<NA>,5F88D510670063B20E789E6453BC2F3FB0BB67F7,<NA>,False,<NA>,9.15,<NA>,5.68,<NA>,23.13
2,2,0.0,0.0,0.0,VIDRACCO,BFAEB566B3BB78B9ABD4F86DB3A78BFD8489013B,FEBE7F29FB854C477AFA7073C70F0E6EA81C6B00,M,53,4F053D54B9F1B6FE565498E7FA726501FD8FDD18,...,<NA>,D4B8928760E729127EB4EB532C86127325FBA468,<NA>,False,1.24,<NA>,-10.13,<NA>,-7.13,<NA>
3,3,0.0,0.0,0.0,MEZZOLOMBARDO,CBD30C382C72ED4A3D9DC9D11C13C155930E7C66,9220A3E83BDCE622F70A1456C7898687FD3EDAC8,M,50,115C7D34A79C66AFCDB4EB65262595B197B0F861,...,"1,08 €/smc",BD4DEF66D7EF2D3D683D9C381D2359AD24B41D9C,<NA>,False,-0.88,<NA>,23.68,<NA>,17.66,<NA>
4,4,0.0,0.0,0.0,PINETO,4D17C711CADED6C1E9BD459088B4B80F24575FE7,642C21354EF6017D13220517E2A25D4A0442D4C5,M,65,C7734B97A212004CEA25956D8FDFEF068CD2B8F6,...,"0,86 €/smc",C0622AFD22384A4B54CD1ACAF7D988B89853A5AC,<NA>,False,14.71,<NA>,141.57,<NA>,63.59,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,26.0,25.0,33.0,CINTANO,2D244E5D90065985546AEA4D52F54C9705B8038B,102CADC9FC8C745FE3B8C27F7267E07C9691728A,M,55,6D3FDD0A2EEEE1A2C1AC23E6C316048DD2CCE859,...,<NA>,53D29ACDEB11F45D4F305A4D00E5F03C1F2CB674,monthly,False,<NA>,13.08,<NA>,7.62,<NA>,15.65
1996,1996,71.0,61.0,88.0,TREZZONE,A01243282C24F561D6B54F1C1FCB45AE80F75B0A,9C3B5B4093FC99569E5BBA918A977681A46F52DF,M,48,52315237533E1A6504E3795EAC1D5586744C62FD,...,<NA>,2B8D02A2AD117EC401BCEF3C0651FEEBB6609D1C,monthly,False,<NA>,9.2,<NA>,8.76,<NA>,25.8
1997,1997,37.0,71.0,100.0,RIPATRANSONE,469AB252FF0A2C60B32D0337C933E3565E88C0C2,1B451B6AA312EFF93372D44EC6C24C0110A628A1,F,32,2777855F19667FBFEDEAEF181E7ABC8584262127,...,<NA>,A1BFA0E4833E27AB398F4420414E037D250F0A7E,monthly,False,<NA>,8.7,<NA>,8.66,<NA>,24.14
1998,1998,74.0,78.0,92.0,MONSELICE,9E24BB26322BD95D11E26DFB80AA1168C9661FDC,F6BADD2456E283A8DCA6F3FF640724072108BAEF,F,54,8DC8820FB50005ED9CC4E7C187B9703460941856,...,<NA>,DED078288677E4E8A15874405E79C26AC78747B0,monthly,False,<NA>,10.21,<NA>,8.96,<NA>,30.43


In [100]:
# No duplicates found on the rows, what about columns?
base.get_duplicate_columns()

[('date', 'light_start_date'),
 ('date', 'gas_end_date'),
 ('light_start_date', 'gas_end_date'),
 ('light_end_date', 'gas_start_date'),
 ('gas_average_cost', 'average_unit_gas_cost')]

In [ ]:
# The story changes here, we got these pairs as candidates as duplicate columns 
# (those which have same values for each row.)

In [101]:
compariser = cudf.Series(np.random.randn(2000))
print(df['gas_average_cost'])
print(df['average_unit_gas_cost'])
print(compariser.shape)
compariser = (df['gas_average_cost'] == df['average_unit_gas_cost']).astype(int)
print(compariser.sum())

<dask_cudf.Series | 15 tasks | 5 npartitions>
<dask_cudf.Series | 15 tasks | 5 npartitions>
(2000,)
dd.Scalar<series-..., dtype=int64>


In [ ]:
# LOCATE OUTLIERS

In [229]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import dask_cudf
import time

cluster = LocalCUDACluster(
    CUDA_VISIBLE_DEVICES="0",
    rmm_pool_size=parse_bytes("150GB"), # This GPU has 16GB of memory
    device_memory_limit=parse_bytes("100GB"),
)
client = Client(cluster)

time.sleep(2)
print("cluster status ", cluster.status)
print("cluster information ", cluster)
print("client information ",client)
client

/opt/conda/envs/rapids/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45567 instead
  f"Port {expected} is already in use.\n"


cluster status  Status.running
cluster information  LocalCUDACluster(76edbb4c, 'tcp://127.0.0.1:42453', workers=1, threads=1, memory=187.40 GiB)
client information  <Client: 'tcp://127.0.0.1:42453' processes=1 threads=1, memory=187.40 GiB>


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:45567/status,
Dashboard: http://127.0.0.1:45567/status,Workers: 1
Total threads: 1,Total memory: 187.40 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42453,Workers: 1
Dashboard: http://127.0.0.1:45567/status,Total threads: 1
Started: Just now,Total memory: 187.40 GiB
Comm: tcp://127.0.0.1:36067,Total threads: 1
Dashboard: http://127.0.0.1:33725/status,Memory: 187.40 GiB
Nanny: tcp://127.0.0.1:43101,


In [230]:
ddf = dask_cudf.read_csv("/data/invoices.csv", chunksize="0.1 GiB")
print(ddf.npartitions)
ddf = ddf.quantile(q=0.5)
_ = wait(ddf)

48


In [223]:
!nvidia-smi

Thu Feb 10 16:45:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 460.27.04    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:65:00.0 Off |                    0 |
| N/A   80C    P0    35W /  70W |  15095MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [231]:
ddf.compute()

MemoryError: std::bad_alloc: CUDA error at: /opt/conda/envs/rapids/include/rmm/mr/device/cuda_memory_resource.hpp

In [ ]:
def column_outliers(x:cudf.Series, k:float):
    q1 = x.quantile(q=0.25)
    q3 = x.quantile(q=0.75)
    I = q3 - q1
    return (x < q1 - k*I) or (x > q3 + k*I)

def df_outliers(df:cudf.DataFrame, k:float):
    q1 = df.quantile(q=0.25)
    q3 = df.quantile(q=0.75)
    I = q3 - q1
    return (x < q1 - k*I) or (x > q3 + k*I)

In [138]:
col_outliers(df.iloc[:,1], 0)

dd.Scalar<quantil..., dtype=float64>

In [164]:
type = ddf.dtypes
num = type[type!='object'][type!='bool']
num.index

Index(['bill_id', 'F1_kWh', 'F2_kWh', 'F3_kWh', 'age', 'light_average_cost',
       'gas_consumption', 'light_consumption', 'gas_offer', 'light_offer',
       'billing_frequency', 'gas_system_charges', 'light_system_charges',
       'gas_material_cost', 'light_transport_cost', 'gas_transport_cost',
       'light_material_cost'],
      dtype='object')

In [193]:
num_df = ddf[num.index].quantile(q=0.2)
num_df = num_df.persist()
num_df.compute()

MemoryError: std::bad_alloc: CUDA error at: /opt/conda/envs/rapids/include/rmm/mr/device/cuda_memory_resource.hpp

In [184]:
# SORT DATA